# Overview

1. load document
2. split
3. embed using a model
4. store in vector store
5. retrieve
6. genration

## environment

In [2]:
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

LANGSMITH_API_KEY  = os.getenv("LANGSMITH_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [11]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGSMITH_API_KEY
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

# 1. Indexing

In [5]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(blog_docs)

In [12]:
# Embed
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"))

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2. RETRIEVAL and GENERATION

In [13]:
# Prompt template
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash")

In [15]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
#Demo
query = "What is Task Decomposition?"
print(rag_chain.invoke(query))

Task decomposition is breaking down a complex task into smaller, simpler sub-tasks.  This can be achieved through prompting techniques like Chain of Thought, which encourages step-by-step reasoning, or by using external planners.  The goal is to make the overall task more manageable.
